# Descriptive Statistics (Chapter 3)

Descriptive statistic normally applies to the analyzing data in a way that describes, shows, or summarizes data in a meaningful way. **We do not draw conclusions from descriptive statistics**!!  That is the art of inferential statistics, which we will explore in the next chapter.  Let us differentiate the two:
* Suppose that we take a sample of everyone's age in the class
* We can use **descriptive statistics** to summarize the data that is collected (mean, median, standard deviation, quantiles, ...)
* We would use **inferential statistics** to infer information about graduate students enrolled in a masters program at Michigan Tech
Often, we want to infer information about a **population**, but are only able to observe a part of the population, known as the **Sample**.

Goal for this module:
* Learning statistical terms and concepts to describe a data sample.
* Using Python to recover descriptive statistics of data
* More experience visualizing data using Python

I will cover a separate example with roughly the same topics/layout as our textbook, so that you have two distinct examples to work from.  The data set I have chosen to use is the wine quality data set from the UCI Machine Learning Repository, https://archive.ics.uci.edu/ml/datasets/Wine+Quality.  As we will see, this data set provides many attributes of wine, including a summary "quality score" based on an expert opinion.  We may visit this data set in a later week to try and use machine learning to assign a quality score based on wine attributes.  For now, we are just interested in generating **descriptive statistics** from the data.  First, lets setup our python libraries and toolboxes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Next, we need to prepare our data.  For this notebook, we will pull our data directly from the UCI repository.  It turns out, there are two data sets: one for red one, one for white wine. Lets pull the data for the redwine.

In [ ]:
urlurl  ==  "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv""https: 
wine = pd.read_csv(url)